In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
import warnings
warnings.filterwarnings("ignore")

In [2]:
df= pd.read_csv('news.csv')

In [3]:
df.shape

(6335, 4)

In [4]:
df['title'].nunique()

6256

In [5]:
df.head(10)

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
5,6903,"Tehran, USA","\nI’m not an immigrant, but my grandparents ...",FAKE
6,7341,Girl Horrified At What She Watches Boyfriend D...,"Share This Baylee Luciani (left), Screenshot o...",FAKE
7,95,‘Britain’s Schindler’ Dies at 106,A Czech stockbroker who saved more than 650 Je...,REAL
8,4869,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,REAL
9,2909,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,REAL


In [6]:
df.isnull().any()

Unnamed: 0    False
title         False
text          False
label         False
dtype: bool

In [7]:
df.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [8]:
df.drop_duplicates('text').shape

(6060, 4)

In [9]:
df['text'].nunique()

6060

In [10]:
df['label'].value_counts()

REAL    3171
FAKE    3164
Name: label, dtype: int64

In [11]:
labels=df.label
labels.head(5)

0    FAKE
1    FAKE
2    REAL
3    FAKE
4    REAL
Name: label, dtype: object

In [13]:
from sklearn.preprocessing import LabelEncoder

label = ['FAKE', 'REAL']
le= LabelEncoder()
le = le.fit(label)
df['label'] = le.fit_transform(df['label'].astype(str))

df.head(10)

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",0
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,0
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,1
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",0
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,1
5,6903,"Tehran, USA","\nI’m not an immigrant, but my grandparents ...",0
6,7341,Girl Horrified At What She Watches Boyfriend D...,"Share This Baylee Luciani (left), Screenshot o...",0
7,95,‘Britain’s Schindler’ Dies at 106,A Czech stockbroker who saved more than 650 Je...,1
8,4869,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,1
9,2909,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,1


In [14]:
X_train,X_test,y_train,y_test = train_test_split(df['text'], labels,test_size=0.2,random_state=0)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(5068,)
(1267,)
(5068,)
(1267,)


In [15]:
# PassiveAggressiveClassifier
tfidf_vectorizer= TfidfVectorizer(stop_words='english', max_df=0.7)
tfidf_train= tfidf_vectorizer.fit_transform(X_train)
tfidf_test= tfidf_vectorizer.transform(X_test)
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train, y_train)

PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
                            loss='hinge', max_iter=50, n_iter_no_change=5,
                            n_jobs=None, random_state=None, shuffle=True,
                            tol=0.001, validation_fraction=0.1, verbose=0,
                            warm_start=False)

In [16]:
y_pred=pac.predict(tfidf_test)
score=accuracy_score(y_test, y_pred)
print(f'Accuracy: {round(score*100, 2)}%')

Accuracy: 93.13%


In [20]:
from sklearn.ensemble import AdaBoostClassifier
adc=AdaBoostClassifier(n_estimators=5,learning_rate=1)
model= adc.fit(tfidf_train,y_train)
adc_predict= model.predict(tfidf_test)
print(accuracy_score(adc_predict, y_test))
print(classification_report(adc_predict,y_test))
print(confusion_matrix(y_test, adc_predict))

0.7963693764798737
              precision    recall  f1-score   support

        FAKE       0.80      0.78      0.79       631
        REAL       0.79      0.81      0.80       636

    accuracy                           0.80      1267
   macro avg       0.80      0.80      0.80      1267
weighted avg       0.80      0.80      0.80      1267

[[494 121]
 [137 515]]


In [21]:
from xgboost import XGBClassifier
xgb=XGBClassifier()
model=xgb.fit(tfidf_train,y_train)
xgb_predict=model.predict(tfidf_test)
print(accuracy_score(xgb_predict,y_test))
print(classification_report(xgb_predict,y_test))
print(confusion_matrix(y_test, xgb_predict))

[19:47:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.9329123914759274
              precision    recall  f1-score   support

        FAKE       0.93      0.93      0.93       612
        REAL       0.94      0.93      0.93       655

    accuracy                           0.93      1267
   macro avg       0.93      0.93      0.93      1267
weighted avg       0.93      0.93      0.93      1267

[[571  44]
 [ 41 611]]
